## Micropython memory map visualizer


In [1]:
from colorama import Fore, Back, Style
import re
from dataclasses import dataclass
import copy

In [2]:
@dataclass
class MemoryInfo:
    """MicroPython Visual Memory Information Map"""

    total: int = 0
    """Total memory"""
    used: int = 0
    """Used memory"""
    free: int = 0
    """Free memory"""
    one_blocks: int = 0
    """Number of 1-blocks"""
    two_blocks: int = 0
    """Number of 2-blocks"""
    max_block_size: int = 0
    """Largest available block"""
    max_free_size: int = 0
    """largest free block"""
    memory_map: str = ""
    """Memory map showing allocated and free areas"""
    memory_map_2: str = ""
    """Memory map showing allocated and free areas"""
    _raw_map: str = ""
    _raw_map_2: str = ""
    lines_free: int = 0
    show_free: bool = True
    columns: int = 2
    rainbow: bool = False
    is_diff = False

    COL_WIDTH = 64

    def __init__(self, mem_info, show_free=False, columns=4, rainbow: bool = False):
        # sourcery skip: use-named-expression
        """Parse the memory map"""

        if issubclass(type(mem_info), list):
            mem_info = "\n".join(mem_info)
        elif issubclass(type(mem_info), str):
            mem_info = str(mem_info)

        self.show_free = show_free
        self.columns = columns
        self._color_num = 0

        match_1 = re.search(r"GC: total: (\d+), used: (\d+), free: (\d+)", mem_info)
        if not match_1:
            raise ValueError("Not recognized as a valid Micropython memory info")
        self.total, self.used, self.free = [int(x) for x in match_1.groups()]
        # find the used blocks
        match_2 = re.search(r" No. of 1-blocks: (\d+), 2-blocks: (\d+), max blk sz: (\d+), max free sz: (\d+)", mem_info)
        if match_2:
            self.one_blocks, self.two_blocks, self.max_block_size, self.max_free_size = [int(x) for x in match_2.groups()]
        match_3 = re.search(r"\((.*) lines all free\)", mem_info)
        if match_3:
            self.lines_free = int(match_3.groups(0)[0])

        self._raw_map = re.findall(r"^[0-9a-fA-F]*\: (.*)", mem_info, flags=re.MULTILINE)
        self.memory_map = "".join(self._raw_map)
        self.rainbow = rainbow

        self.update_map()

    def update_map(self):
        "show/hide the free space in the memory map"
        _map = self._raw_map.copy()
        # TODO: Where to insert the free lines - currently just use -1 ...
        if self.show_free:
            for _ in range(self.lines_free):
                _map.insert(-1, "." * 64)
        self.memory_map = "".join(_map)

    def _repr_pretty_(self, pp, cycle):
        if not self.is_diff:
            return self._repr_pretty_memmap_(pp, cycle)
        else:
            return self._repr_pretty_diff_(pp, cycle)

    def _repr_pretty_memmap_(self, pp, cycle):
        "print a colored version of the memory map"
        width = self.COL_WIDTH * self.columns
        text = (
            f"{Fore.WHITE}{Back.BLACK}Memory Used: 0x{self.used:X} of Total: 0x{self.total:X}\n"
            f"Free: 0x{self.free:X} {self.free/self.total:.1%}\n"
        )

        color = Fore.WHITE
        for i in range(len(self.memory_map)):
            # '=' keeps the same color
            if self.memory_map[i] != "=":
                color = self.color(self.memory_map[i])
            text += color + self.memory_map[i]
            # columns
            if (i + 1) % self.COL_WIDTH == 0:
                text += f"{Style.RESET_ALL} "
            # rows
            if (i + 1) % width == 0:
                text += Style.RESET_ALL + "\n"
        # now pretty print the memory map
        pp.text(text)

    def _repr_pretty_diff_(self, pp, cycle):
        "print a colored version of a differential memory map"
        width = self.COL_WIDTH * self.columns
        text = (
            # f"{Fore.WHITE}{Back.BLACK}Memory Used: 0x{self.used:X} of Total: 0x{self.total:X}\n"
            f"Free: 0x{self.free:X} {self.free/self.total:.1%}\n"
        )

        color = Fore.WHITE
        for i in range(len(self.memory_map)):
            # '=' keeps the same color
            if self.memory_map[i] != "=":
                other = self.memory_map_2[i] if i < len(self.memory_map_2) else ""
                color = self.diff_color(self.memory_map[i], other)
            text += color + self.memory_map[i]
            # columns
            if (i + 1) % self.COL_WIDTH == 0:
                text += f"{Style.RESET_ALL} "
            # rows
            if (i + 1) % width == 0:
                text += Style.RESET_ALL + "\n"
        pp.text(text)

    def color(self, c: str):
        # ====== =================
        # Symbol Meaning
        # ====== =================
        #    .   free block
        #    h   head block
        #    =   tail block
        #    m   marked head block
        #    T   tuple
        #    L   list
        #    D   dict
        #    F   float
        #    B   byte code
        #    M   module
        #    S   string or bytes
        #    A   bytearray
        # ====== =================
        BG_COLORS = [Back.BLUE, Back.RED, Back.MAGENTA, Back.CYAN]
        fg = Fore.BLACK
        bg = Back.RED
        if c == ".":
            fg = Fore.GREEN
            bg = Back.GREEN
        elif c.isupper():
            fg = Fore.WHITE
        else:
            fg = Fore.BLACK
        if c in "TSLDFABh":
            if self.rainbow:
                bg = BG_COLORS[self._color_num]
                self._color_num = (self._color_num + 1) % len(BG_COLORS)
            else:
                bg = Back.RED
        elif c == "M":
            fg = Fore.BLACK
            bg = Back.CYAN
        return fg + bg

    def diff_color(self, c: str, other: str):
        "Colors for the diff view"
        if c == other:
            # same
            fg = Fore.BLACK
            bg = Back.MAGENTA
        elif c == ".":
            # freed up
            fg = Fore.GREEN
            bg = Back.GREEN
        elif c.isupper():
            # allocated Caps
            fg = Fore.YELLOW
            bg = Back.RED
        else:
            # allocated lower
            fg = Fore.WHITE
            bg = Back.RED

        return fg + bg

    def __sub__(self, other):
        # assume self is the newer / larger memory info
        diff = copy.deepcopy(self)
        diff.is_diff = True
        diff.used = self.used - other.used
        diff.free = self.free - other.free
        diff.one_blocks = self.one_blocks - other.one_blocks
        diff.two_blocks = self.two_blocks - other.two_blocks
        # diff.max_block_size = self.max_block_size - other.max_block_size
        # diff.max_free_size = self.max_free_size - other.max_free_size
        # diff.lines_free = self.lines_free - other.lines_free
        # diff.memory_map = self.memory_map
        diff.memory_map_2 = other.memory_map
        diff._raw_map_2 = other._raw_map

        return diff

In [3]:
%mpy --reset
mem_lst = %mpy import micropython; micropython.mem_info(True)
mem_lst


['stack: 516 out of 7936', 'GC: total: 193664, used: 4496, free: 189168', ' No. of 1-blocks: 53, 2-blocks: 14, max blk sz: 64, max free sz: 11813', 'GC memory layout; from 20010b80:', '00000000: h=MhhhhBD.hBTTBDhTBDBBBh===DBDh====B=BBBBBBTB=BTB=BBBTB=TBTB=Bh=', '00000400: ==BB=h===========TB=h=h===================h=====h==h============', '00000800: =====h==========================================================', '00000c00: =====h==========================================================', '00001000: =====h=hhhBhhhhBh=h..h..h=.....h=h=.............................', '       (184 lines all free)', '0002f400: ........']

In [4]:
# show the memory map
mem = MemoryInfo(mem_lst.data)
mem

Memory Used: 0x1190 of Total: 0x2F480
Free: 0x2E2F0 97.7%
h=MhhhhBD.hBTTBDhTBDBBBh===DBDh====B=BBBBBBTB=BTB=BBBTB=TBTB=Bh= ==BB=h===========TB=h=h===================h=====h==h============ =====h========================================================== =====h========================================================== 
=====h=hhhBhhhhBh=h..h..h=.....h=h=............................. ........

In [5]:
# show the memory map with free space expanded
mem = MemoryInfo(mem_lst.data, show_free=True)
mem

Memory Used: 0x1190 of Total: 0x2F480
Free: 0x2E2F0 97.7%
h=MhhhhBD.hBTTBDhTBDBBBh===DBDh====B=BBBBBBTB=BTB=BBBTB=TBTB=Bh= ==BB=h===========TB=h=h===================h=====h==h============ =====h========================================================== =====h========================================================== 
=====h=hhhBhhhhBh=h..h..h=.....h=h=............................. ................................................................ ................................................................ ................................................................ 
................................................................ ................................................................ ................................................................ ................................................................ 
................................................................ ................................................................ .............................

In [6]:
# show the memory map in a single column - similar to the micropython.mem_info() output
mem = MemoryInfo(mem_lst.data, columns=1)
mem

Memory Used: 0x1190 of Total: 0x2F480
Free: 0x2E2F0 97.7%
h=MhhhhBD.hBTTBDhTBDBBBh===DBDh====B=BBBBBBTB=BTB=BBBTB=TBTB=Bh= 
==BB=h===========TB=h=h===================h=====h==h============ 
=====h========================================================== 
=====h========================================================== 
=====h=hhhBhhhhBh=h..h..h=.....h=h=............................. 
........

In [7]:
# consume some more memory

%mpy print(help('modules'))
# %mpy import gc; gc.collect()

mem_lst = %mpy import micropython; micropython.mem_info(True)
mem_2=MemoryInfo(mem_lst.data, columns=1 )
mem_2



Memory Used: 0x1590 of Total: 0x2F480
Free: 0x2DEF0 97.1%
h=MhhhhBDhhBTTBDhTBDBBBh===DBDh====B=BBBBBBTB=BTB=BBBTB=TBTB=Bh= 
==BB=h===========TB=h=h===================h=====h==h============ 
=====h========================================================== 
=====h========================================================== 
=====h=hhhBhhhhBh=hhhhhhh=BLShSh=h=hSh=ShShSh==h===============h 
ShShShShShShShShhhhB..h........h=..h=........................... 
........

In [8]:
# %%micropython
# eat even more memory
import gc

gc.disable()

foo = 2
for _ in range(500):
    foo = foo + foo

[]

In [9]:
# take anoter memory snapshot 
mem_lst = %mpy micropython.mem_info(True)
mem_3=MemoryInfo(mem_lst.data)

mem_3


Memory Used: 0x90D0 of Total: 0x2F480
Free: 0x263B0 80.9%
h=MhhhhBDhhBTTBDhTBDBBBh===DBDh====B=BBBBBBTB=BTB=BBBTB=TBTB=Bh= ==BB=h===========TB=h=h===================h=====hhhh============ =====h========================================================== =====h========================================================== 
=====h=hhhBhhhhBh=hhhhhhh=BLShSh=h=hSh=ShShSh==h===============h ShShShShShShShShhhhBhhhhBhhhhhSh=hhh=Bhhhhh=hh===Bhhhhh=h=BShhhB hhhhh=hhhhhhhhh=======h========h================================ =======================================h===h=======h====h==hhhhh 
hhhhhhhh=======h================================================ =======================h===hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh=hh=hh=hh=hh=hh=hh=hh=h 
h=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh =hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh= hh=hh=hh=hh=hh=hh=hh=hh=hh=hh

In [14]:
# compare snapshot 1 and 2 to see
#  - what remained the same = Black on Blue
#  - what was freed up = Green on Green ( can also be due to additional pages being reported)
#  - what was allocated = Yellow or White on Red

mem_2 - mem

Memory Used: 0x400 of Total: 0x2F480
Free: 0x-400 -0.5%
h=MhhhhBDhhBTTBDhTBDBBBh===DBDh====B=BBBBBBTB=BTB=BBBTB=TBTB=Bh= 
==BB=h===========TB=h=h===================h=====h==h============ 
=====h========================================================== 
=====h========================================================== 
=====h=hhhBhhhhBh=hhhhhhh=BLShSh=h=hSh=ShShSh==h===============h 
ShShShShShShShShhhhB..h........h=..h=........................... 
........

In [ ]:
# compare snapshot 2 and 3
mem_3 - mem_2

Memory Used: 0x7B40 of Total: 0x2F480
Free: 0x-7B40 -16.3%
h=MhhhhBDhhBTTBDhTBDBBBh===DBDh====B=BBBBBBTB=BTB=BBBTB=TBTB=Bh= ==BB=h===========TB=h=h===================h=====hhhh============ =====h========================================================== =====h========================================================== 
=====h=hhhBhhhhBh=hhhhhhh=BLShSh=h=hSh=ShShSh==h===============h ShShShShShShShShhhhBhhhhBhhhhhSh=hhh=Bhhhhh=hh===Bhhhhh=h=BShhhB hhhhh=hhhhhhhhh=======h========h================================ =======================================h===h=======h====h==hhhhh 
hhhhhhhh=======h================================================ =======================h===hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh=hh=hh=hh=hh=hh=hh=hh=h 
h=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh =hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh= hh=hh=hh=hh=hh=hh=hh=hh=hh=h

In [11]:
# now free up some memory
%mpy import gc; gc.collect()

# take another memory snapshot 
mem_lst = %mpy import micropython; micropython.mem_info(True)
mem_4=MemoryInfo(mem_lst.data )

# ad show the difference between the last two snapshots
mem_4-mem_3

Memory Used: 0x-7A30 of Total: 0x2F480
Free: 0x7A30 16.2%
h=Mhhhh=DhhhhhBDh..DBBBh===DBDh====B=BBBBBB.B=B.B=BBB.B=.B.B=Bh= ==BB=h===========.B=h=h===================h=====...h============ =====h========================================================== =====h========================================================== 
=====.....h=..h=h=.............................................. .........................h...................................... ..............h=======h========h================================ =======================================............h====........ 
...............................................................h h====....hh..B..................h=.............................. ........

In [15]:
# or the reverse to see what got freed ( on RED)
mem_3 - mem_4

Memory Used: 0x7A30 of Total: 0x2F480
Free: 0x-7A30 -16.2%
h=MhhhhBDhhBTTBDhTBDBBBh===DBDh====B=BBBBBBTB=BTB=BBBTB=TBTB=Bh= ==BB=h===========TB=h=h===================h=====hhhh============ =====h========================================================== =====h========================================================== 
=====h=hhhBhhhhBh=hhhhhhh=BLShSh=h=hSh=ShShSh==h===============h ShShShShShShShShhhhBhhhhBhhhhhSh=hhh=Bhhhhh=hh===Bhhhhh=h=BShhhB hhhhh=hhhhhhhhh=======h========h================================ =======================================h===h=======h====h==hhhhh 
hhhhhhhh=======h================================================ =======================h===hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh=hh=hh=hh=hh=hh=hh=hh=h 
h=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh =hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh= hh=hh=hh=hh=hh=hh=hh=hh=hh=h